### Comparing the difference in edges with other networks

Comparing with the naive networks (mild vs severe) to see if they agree with the differential network results:

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import sspa

In [ ]:
mild = nx.read_gml("Cytoscape/metabolomic_final_mildcases.gml")
mild_nodes = list(mild.nodes())

severe = nx.read_gml("Cytoscape/metabolomic_final_severecases.gml")
severe_nodes = list(severe.nodes())

G = nx.read_gml("Cytoscape/metabolomic_final_commoncases.gml")
all_edges = list(G.edges())
all_nodes = list(G.nodes())
len(all_edges)

In [ ]:
mild = nx.read_gml("Cytoscape/proteomic_final_mildcases.gml")
mild_nodes = list(mild.nodes())

severe = nx.read_gml("Cytoscape/proteomic_final_severecases.gml")
severe_nodes = list(severe.nodes())

G = nx.read_gml("Cytoscape/proteomic_final_commoncases.gml")
all_edges = list(G.edges())
all_nodes = list(G.nodes())
len(all_edges)

80969

In [ ]:
mild = nx.read_gml("Cytoscape/integrated_final_mildcases.gml")
mild_nodes = list(mild.nodes())

severe = nx.read_gml("Cytoscape/integrated_final_severecases.gml")
severe_nodes = list(severe.nodes())

G = nx.read_gml("Cytoscape/integrated_final_commoncases.gml")
all_edges = list(G.edges())
all_nodes = list(G.nodes())
len(all_edges)

In [ ]:
#Find nodes which are present in mild/severe network not present in full network
set((mild_nodes)).difference(set(all_nodes).intersection(set(mild_nodes)))

In [ ]:
#Confirm that the mild and severe nodes DO NOT have new nodes that the full network doesn't
print(len(all_nodes))
print(len(mild_nodes))
print(len(severe_nodes))
print(len(list(set(all_nodes).intersection(set(mild_nodes)))))
print(len(list(set(all_nodes).intersection(set(severe_nodes)))))

#Metabolomic severe network has 3 new nodes  #{'R-HSA-211859', 'R-HSA-5663205', 'R-HSA-9734207'}
#Proteomic same nodes for all
#Integrated mild network has 4 new nodes and severe network has 3 new nodes  #{'R-HSA-192105', 'R-HSA-193368', 'R-HSA-211976', 'R-HSA-5619084'} {'R-HSA-174824', 'R-HSA-83936', 'R-HSA-8956321'} 

In [ ]:
mild_edges = list(mild.edges())
print(len(mild_edges))
severe_edges = list(severe.edges())
print(len(severe_edges))

7333
71172


In [ ]:
#Example code from https://stackoverflow.com/questions/41125909/python-find-elements-in-one-list-that-are-not-in-the-other
list_1=["a", "b", "c", "d", "e"]
list_2=["a", "f", "c", "m"]
set(list_2) - set(list_1)

#set(['m', 'f'])

In [ ]:
#Edges present in the mild network but not the severe network
print(len(list(set(mild_edges) - set(severe_edges))))
mild = list(set(mild_edges) - set(severe_edges))

In [ ]:
#Edges present in the severe network but not the mild network
print(len(list(set(severe_edges) - set(mild_edges))))
severe = list(set(severe_edges) - set(mild_edges))

In [ ]:
#Edges present in both mild and severe
print(len(list(set(severe_edges) & set(mild_edges))))
mild_severe = list(set(severe_edges) & set(mild_edges))

In [ ]:
print(len(list(set(mild) - set(all_edges)))) #Edges in the mild network (but not severe network) not in the full network

print(len(list(set(severe) - set(all_edges)))) #Edges in the severe network (but not mild network) not in the full network

print(len(list(set(mild_severe) - set(all_edges)))) #Edges in the mild network AND severe network not in the full network

In [ ]:
mild_dict = {mild[i]:"Mild" for i in range(0,len(mild))}
severe_dict = {severe[i]:"Severe" for i in range(0,len(severe))}

condition_dict = mild_dict.copy()
for key, value in severe_dict.items():
    condition_dict[key] = value

print(condition_dict)

In [ ]:
G=nx.from_edgelist(mild+severe)
len(G.edges())
nx.set_edge_attributes(G, condition_dict, "Condition")

In [ ]:
#IF YOU DON'T REMOVE THE EDGES, DON'T RUN THIS CODE

#Add the betweenness centrality as a node attribute
betweenness= nx.betweenness_centrality(G, normalized=True) #output as dictionary
#display(betweenness)
nx.set_node_attributes(G, betweenness, "betweenness")

#Add the betweenness centrality as a node attribute
degree= nx.degree_centrality(G) #output as dictionary
#display(degree)
nx.set_node_attributes(G, degree, "degrees")


#Check
print(G.number_of_nodes()) 
#G.nodes['R-HSA-110331']#["betweenness"]

In [ ]:
#Using Cecilia's code

#Creating graph
hierarchy = pd.read_csv('Data/ReactomePathwaysRelation.txt', sep='\t', header=None)

#From the pathways, subset to Homo sapiens only
hierarchy_hsa = hierarchy[hierarchy[0].str.contains('HSA')]

#Return unique values in the first column that is not in the second column as a numpy array
#These values are not child pathways in any instances
hierarchy_hsa_parents = np.setdiff1d(hierarchy_hsa[0], hierarchy_hsa[1])

#Add the unique values not in the second column as a second attached dataset to the bottom of the original data
#The first column represents the parent column, the second column is the child column
hierarchy_hsa_all = pd.concat([hierarchy_hsa, pd.DataFrame([hierarchy_hsa_parents, hierarchy_hsa_parents], index=[0, 1]).T])

#DiGraph is a directed graph
H = nx.from_pandas_edgelist(hierarchy_hsa, source=0, target=1, create_using=nx.DiGraph())


In [ ]:
#Convert pathway ID to name
root_path = pd.read_excel('Data/Root_pathways.xlsx', header=None)
root_pathway_dict = {root_path[0][i]:root_path[1][i] for i in range(0,len(root_path))}

#Using Cecilia's code

#Find the root pathway

def find_root(H,child):
    #Find parent from child 
    parent = list(H.predecessors(child))

    #Keep the loop going until the highest level is reached
    if len(parent) == 0:
        return child
    else:  
        return find_root(H, parent[0])

hierarchy_hsa_all['Root'] = [find_root(H, i) for i in hierarchy_hsa_all[1]]

hierarchy_hsa_all.columns = ['Parent', 'Child', 'Root']

#There are 83 instances of duplicates, however all the child duplicates have the same root (even though different parents) after checking

root_pathways = {}
for pathway in list(H.nodes):
    index = hierarchy_hsa_all.Child[hierarchy_hsa_all.Child == pathway].index.tolist()[0]
    root_pathway  = hierarchy_hsa_all.Root[index]
    label = root_pathway_dict[root_pathway]
    root_pathways[pathway] = label


In [ ]:
#Using Cecilia's code
 
#Shows all the root pathways in Reactome
set(hierarchy_hsa_all['Root'] )
#Shows all the root pathways present in the original dataset
set(root_pathways.values())

nx.set_node_attributes(G, root_pathways, "root_pathway")

print(G.number_of_nodes()) 

In [ ]:
#Assign pathway name to node

#Metabolomic:
#reactome_pathways = sspa.process_gmt("Data/Reactome_Homo_sapiens_pathways_compounds_R84.gmt") 

#Proteomic:
#reactome_pathways = sspa.process_reactome('Homo sapiens', infile = 'Data/UniProt2Reactome_All_Levels_ver84.txt', download_latest = False, filepath = None)

#Integrated:
reactome_pathways = pd.read_csv("Data/Reactome_multi_omics_ChEBI_Uniprot.csv", index_col=0,dtype="str")


In [ ]:

pathway_name_dict = {reactome_pathways.index[i]:reactome_pathways["Pathway_name"][i] for i in range(0,len(reactome_pathways))}
pathway_name_dict['R-HSA-1483257']
pathway_name_dict = {k:pathway_name_dict[k] for k in list(G.nodes)}

nx.set_node_attributes(G, pathway_name_dict, "pathway_name")

In [ ]:
#nx.write_gml(G,'Cytoscape/metabolomic_condition_diff.gml')
#nx.write_gml(G,'Cytoscape/proteomic_condition_diff.gml')
nx.write_gml(G,'Cytoscape/integrated_condition_diff.gml')

In [ ]:
#Reading in the edges expressed in the differential network
#with open('Data/permutation_test_files_metabolomics/sig_edges_beforefdr.txt') as f:
with open('Data/permutation_test_files_metabolomics/sig_edges.txt') as f:
#with open('Data/permutation_test_files_proteomics/sig_edges.txt') as f:
#with open('Data/permutation_test_files_integrated/sig_edges.txt') as f:
    lines = f.readlines()

edges_remaining = []

edges = lines[0].split(",")

for index in range(0,len(edges),2):
    list1 = edges[index],(edges[index+1][1:]) #becomes a tuple
    edges_remaining.append(list1)

print(len(edges_remaining))
#edges_remaining

In [ ]:
intersection = list(set(edges_remaining).intersection(list(set(mild))))  
len(intersection)

In [ ]:
intersection = list(set(edges_remaining).intersection(list(set(severe)))) 
len(intersection) 

In [ ]:
intersection = list(set(edges_remaining).intersection(list(set(mild_severe)))) 
len(intersection) 

In [ ]:
all_edges

In [ ]:
edges_remaining

In [ ]:
intersection = list(set(edges_remaining).intersection(list(set(all_edges)))) 
len(intersection) 

### Comparing with the significant edges BETWEEN the different omics:

In [ ]:
def edge_num (omics):
    with open('Data/permutation_test_files_'+omics+'/sig_edges.txt') as f:
    #with open('Data/permutation_test_files_'+omics+'/sig_edges_beforefdr.txt') as f:    #for metabolomic data since there are so few edges left
        lines = f.readlines()
    edges_remaining = []
    edges = lines[0].split(",")

    for index in range(0,len(edges),2):
        list1 = edges[index],(edges[index+1][1:]) #becomes a tuple
        edges_remaining.append(list1)

    return edges_remaining


In [ ]:
proteomic_edges = edge_num('proteomics')
integrated_edges = edge_num('integrated')
metabolomic_edges = edge_num('metabolomics')

In [ ]:
len(integrated_edges)

In [ ]:
intersection = list(set(integrated_edges).intersection(list(set(metabolomic_edges))))  
len(intersection) 

In [ ]:
intersection = (set(metabolomic_edges) & set(proteomic_edges) & set(integrated_edges))  
len(intersection) 

In [ ]:
len(list(set(integrated_edges) - (set(metabolomic_edges)|set(proteomic_edges)))) #Pathway pairs for integrated data not detected by either single omics

### Get number of common pathways between the three omics

In [ ]:
import pandas as pd
import sspa

In [ ]:
metabolomic_df = pd.read_csv('Data/Su_COVID_metabolomics_processed_commoncases.csv', index_col=0)
proteomic_df = pd.read_csv('Data/Su_COVID_proteomics_processed_commoncases.csv', index_col=0)
integrated_df = pd.read_csv("Data/Su_integrated_data.csv", index_col=0)

#Remove root pathways
#Convert pathway ID to name
root_path = pd.read_excel('Data/Root_pathways.xlsx', header=None)
root_pathway_dict = {root_path[0][i]:root_path[1][i] for i in range(0,len(root_path))}
root_pathway_names = list(root_pathway_dict.keys())

In [ ]:
reactome_pathways = sspa.process_gmt("Data/Reactome_Homo_sapiens_pathways_compounds_R84.gmt")
kpca_scores = sspa.sspa_kpca(metabolomic_df.iloc[:,:-2], reactome_pathways)

#Using Sara's code
kpca_scores = kpca_scores.drop(columns = list(set(root_pathway_names) & set(kpca_scores.columns)))

metabolomic_pathways = kpca_scores.columns
metabolomic_pathways

In [ ]:
#Load in a file downloaded from https://reactome.org/download/current/UniProt2Reactome_All_Levels.txt
reactome_pathways = sspa.process_reactome('Homo sapiens', infile = 'Data/UniProt2Reactome_All_Levels_ver84.txt', download_latest = False, filepath = None)
kpca_scores = sspa.sspa_kpca(proteomic_df.iloc[:,:-2], reactome_pathways)

#Using Sara's code
kpca_scores = kpca_scores.drop(columns = list(set(root_pathway_names) & set(kpca_scores.columns)))

proteomic_pathways = kpca_scores.columns
proteomic_pathways

In [ ]:
reactome_pathways = pd.read_csv("Data/Reactome_multi_omics_ChEBI_Uniprot.csv", index_col=0,dtype="str")
kpca_scores = sspa.sspa_kpca(integrated_df.iloc[:,:-2], reactome_pathways)

#Using Sara's code
kpca_scores = kpca_scores.drop(columns = list(set(root_pathway_names) & set(kpca_scores.columns)))

integrated_pathways = kpca_scores.columns
integrated_pathways

In [ ]:
intersection = list(set(metabolomic_pathways).intersection(list(set(proteomic_pathways))))   #47 pathways
intersection = (set(metabolomic_pathways) & set(proteomic_pathways) & set(integrated_pathways))  #47 pathways 

intersection = list(set(metabolomic_pathways).intersection(list(set(integrated_pathways))))  #144 pathways (same as number of metabolomic)
intersection = list(set(proteomic_pathways).intersection(list(set(integrated_pathways))))  #578 pathways (same as number of proteomic)

#710 pathways in integrated dataset
#((144+578)-47+35)

len(list(set(integrated_pathways) - (set(metabolomic_pathways)|set(proteomic_pathways)))) #35 pathways for integrated data not detected by either single omics

In [ ]:
new_pathways = list(set(integrated_pathways) - (set(metabolomic_pathways)|set(proteomic_pathways)))
new_pathways

In [ ]:
#Get number of pathways that have been influenced

df = pd.read_csv("Data/Su_integrated_data.csv", index_col=0)
reactome_pathways = pd.read_csv("Data/Reactome_multi_omics_ChEBI_Uniprot.csv", index_col=0,dtype="str")


kpca_scores = sspa.sspa_kpca(integrated_df.iloc[:,:-2], reactome_pathways)

#Using Sara's code
kpca_scores = kpca_scores.drop(columns = list(set(root_pathway_names) & set(kpca_scores.columns)))

#Obtain pathways and corresponding metabolites for all Reactome pathways, store as dictionary
orig_dict = sspa.utils.pathwaydf_to_dict(reactome_pathways)

#Filter out dictionary to retain only the pathways that remain after kPCA
my_keys = kpca_scores.columns
pathways_dict = {key: orig_dict[key] for key in my_keys}

#Filter out the compounds in the pathways that are not present in the dataset

#Obtain all unique values in dataset
compounds_present = list(df.columns[:-2])
filtered_dict = {} 

#My code adapted from Cecilia's
#If the key values are not part of the compounds in dataset then remove
for key,value in pathways_dict.items():
    new_val = [item for item in value if item in compounds_present]
    if len(new_val) >= 2: #at least two compounds in the pathway
        filtered_dict[key] = new_val

In [ ]:
new_pathways

In [ ]:
#All 35 new pathways comprised of one metabolite and one protein

proteomic_only_pathway = []
proteomic_pathway_with_onemetabolite = []
metabolomic_only_pathway = []
metabolomic_pathway_with_oneprotein = []
both_omic = []
new_pathway = []

for key,values in filtered_dict.items():
    #if key in new_pathways:
        #print(values)
    #print(values)
    proteomic=False
    metabolomic=False
    both=False


    proteins = 0
    metabolites = 0

    for value in values:

        if value[0].isalpha():
            proteins += 1
        else:
            metabolites += 1

    
    if metabolites == 0:
        proteomic_only_pathway.append(key)
    if proteins > 1 and metabolites == 1:
        proteomic_pathway_with_onemetabolite.append(key)
    
    if proteins == 0:
        metabolomic_only_pathway.append(key)
    if metabolites > 1 and proteins == 1:
        metabolomic_pathway_with_oneprotein.append(key)

    if metabolites > 1 and proteins > 1:
        both_omic.append(key)
    if proteins == 1 and metabolites == 1:
        new_pathway.append(key)



In [ ]:
print(len(proteomic_only_pathway))
print(len(proteomic_pathway_with_onemetabolite))
print(len(metabolomic_only_pathway))
print(len(metabolomic_pathway_with_oneprotein))
print(len(both_omic))
print(len(new_pathway))

In [ ]:
for key,values in filtered_dict.items():
    #if key in new_pathways:
        #print(values)
    print(values)
    proteomic=False
    metabolomic=False
    both=False


    for value in values:

        if value[0].isalpha() and metabolomic==False:
            proteomic=True
        elif value[0].isdigit() and proteomic==False:
            metabolomic=True

        elif value[0].isalpha() and metabolomic==True:
            both=True
            metabolomic=False
            break
        elif value[0].isdigit() and proteomic==True:
            both=True
            proteomic=False
            break

    if proteomic == True:
        proteomic_only.append(key)
    if metabolomic == True:
        metabolomic_only.append(key)
    if both == True:
        both_omic.append(key)


### Comparing the initial test statistics of the three omics to see which edges were affected

In [ ]:
import pandas as pd

Comparing the initial test statistics of the integrated data with the proteomic initial test statistics:

In [ ]:
#Load dataset
metabolomic = pd.read_csv('Data/permutation_test_files_metabolomics/initial_tstats.csv', index_col=0)
proteomic = pd.read_csv('Data/permutation_test_files_proteomics/initial_tstats.csv', index_col=0)
integrated = pd.read_csv('Data/permutation_test_files_integrated/initial_tstats.csv', index_col=0)

In [ ]:
metabolomic = metabolomic.rename(columns={'Initial_tstat': 'metabolomic_tstat'})
proteomic = proteomic.rename(columns={'Initial_tstat': 'proteomic_tstat'})
integrated = integrated.rename(columns={'Initial_tstat': 'integrated_tstat'})

In [ ]:
initial_tstat_comp = proteomic.join(integrated)
initial_tstat_comp

In [ ]:
#Number of pathway pairs with initial test statistic that are the same

proteomic_only_edge = []
metabolomic_influence_edge = []
counter = 0

for edge in initial_tstat_comp.index:
    if initial_tstat_comp.loc[edge].proteomic_tstat == initial_tstat_comp.loc[edge].integrated_tstat:
        proteomic_only_edge.append(edge)
        counter += 1
    else:
        metabolomic_influence_edge.append(edge)


print(counter)

#166,753 edges of the proteomic initial test statistic dataset are all included in the integrated initial test statistic data
#117,381 edges out of 166,753 are identical
#Therefore 49,372 edges were influenced by the metabolomic data

len(metabolomic_influence_edge)

In [ ]:
#Change to tuple format so you can compare with the significant edges

metabolomic_influence_edge_tuple = []
for i in range(len(metabolomic_influence_edge)):
    edge = metabolomic_influence_edge[i].split(",")
    edge = (edge[0],edge[1][1:])
    #edge = tuple(sorted(edge))
    metabolomic_influence_edge_tuple.append(edge)


proteomic_only_edge_tuple  = []
for i in range(len(proteomic_only_edge )):
    edge = proteomic_only_edge [i].split(",")
    edge = (edge[0],edge[1][1:])
    #edge = tuple(sorted(edge))
    proteomic_only_edge_tuple.append(edge)

In [ ]:
integrated_edge = integrated.index

integrated_edge_tuple  = []
for i in range(len(integrated_edge)):
    edge = integrated_edge [i].split(",")
    edge = (edge[0],edge[1][1:])
    #edge = tuple(sorted(edge))
    integrated_edge_tuple.append(edge)



In [ ]:
#Need to see proportion of metabolomic-influenced edges that are significantly differentially expressed edges (from differential network) in the proteomic data vs the integrated data
print(len(metabolomic_influence_edge))
print(len(list(set(proteomic_edges).intersection(set(metabolomic_influence_edge_tuple))))) #2786 edges significant
print(len(list(set(integrated_edges).intersection(set(metabolomic_influence_edge_tuple))))) #After integrating with the metabolomic data, 2425 significant


#1438 edges are the same between 2786 edges and 2425 edges
list1 = list(set(proteomic_edges).intersection(set(metabolomic_influence_edge_tuple)))
list2 = list(set(integrated_edges).intersection(set(metabolomic_influence_edge_tuple)))
len(list(set(list1).intersection(set(list2))))

In [ ]:
#Showing that 'Metabolomic influenced' proteomic pathways are instances where at least one pathway is a common pathway OR one pathway is influenced by one metabolite (not enough to be considered a pathway in the metabolomic set)

list1 = list(set(proteomic_edges).intersection(set(metabolomic_influence_edge_tuple)))

common_pathways = list(set(proteomic_pathways) & set(metabolomic_pathways))
print(len(common_pathways))
counter = 0 

influenced_noncommon_pathways = []
for edges in list1:
    pathway_pair = list(edges)
    if (pathway_pair[0] in common_pathways) or (pathway_pair[1] in common_pathways):
        #print(pathway_pair)
        counter += 1
    else:
        influenced_noncommon_pathways.append(edges)


print(counter)
#1371 pathway pairs have one common pathway in them, out of 49,372 metabolomic influenced protein pathways
#The other cases are where there is one metabolite added to one of the pathways that is not enough to be constituted as a pathway in the metabolomics dataset

In [ ]:
df = pd.read_csv('Data/Su_COVID_metabolomics_processed_commoncases.csv', index_col=0)
reactome_pathways = sspa.process_gmt("Data/Reactome_Homo_sapiens_pathways_compounds_R84.gmt")

#df = pd.read_csv("Data/Su_integrated_data.csv", index_col=0)
#reactome_pathways = pd.read_csv("Data/Reactome_multi_omics_ChEBI_Uniprot.csv", index_col=0,dtype="str")


kpca_scores = sspa.sspa_kpca(integrated_df.iloc[:,:-2], reactome_pathways)

#Using Sara's code
kpca_scores = kpca_scores.drop(columns = list(set(root_pathway_names) & set(kpca_scores.columns)))

#Obtain pathways and corresponding metabolites for all Reactome pathways, store as dictionary
orig_dict = sspa.utils.pathwaydf_to_dict(reactome_pathways)

#Filter out dictionary to retain only the pathways that remain after kPCA
my_keys = kpca_scores.columns
pathways_dict = {key: orig_dict[key] for key in my_keys}

#Filter out the compounds in the pathways that are not present in the dataset

#Obtain all unique values in dataset
compounds_present = list(df.columns[:-2])
filtered_dict = {} 

#My code adapted from Cecilia's
#If the key values are not part of the compounds in dataset then remove
for key,value in pathways_dict.items():
    new_val = [item for item in value if item in compounds_present]
    if len(new_val) >= 2: #at least two compounds in the pathway
        filtered_dict[key] = new_val

In [ ]:
print(influenced_noncommon_pathways)
list(reactome_pathways.loc['R-HSA-422475'])
filtered_dict['R-HSA-422475']

Comparing the initial test statistics of the integrated data with the metabolomic initial test statistics:

In [ ]:
initial_tstat_comp = metabolomic.join(integrated)
initial_tstat_comp

In [ ]:
#Number of pathway pairs with initial test statistic that are the same
metabolomic_only_edge = []
proteomic_influence_edge = []

counter = 0
for edge in initial_tstat_comp.index:
    if initial_tstat_comp.loc[edge].metabolomic_tstat == initial_tstat_comp.loc[edge].integrated_tstat:
        metabolomic_only_edge.append(edge)
        counter += 1
    else:
        proteomic_influence_edge.append(edge)

print(counter)

#10,296 edges of the metabolomic initial test statistic dataset are all included in the integrated initial test statistic data
#2,088 edges out of 10,296 are identical
#Therefore 8,208 edges were influenced by the proteomic data

proteomic_influence_edge

In [ ]:
proteomic_influence_edge_tuple = []
for i in range(len(proteomic_influence_edge)):
    edge = proteomic_influence_edge[i].split(",")
    edge = (edge[0],edge[1][1:])
    #edge = tuple(sorted(edge))
    proteomic_influence_edge_tuple.append(edge)


metabolomic_only_edge_tuple = []
for i in range(len(metabolomic_only_edge)):
    edge = metabolomic_only_edge[i].split(",")
    edge = (edge[0],edge[1][1:])
    #edge = tuple(sorted(edge))
    metabolomic_only_edge_tuple.append(edge)

In [ ]:
#Need to see proportion of proteomic-influenced edges that are significant edges in the metabolomic data vs the integrated data
print(len(proteomic_influence_edge))
print(len(list(set(metabolomic_edges).intersection(set(proteomic_influence_edge_tuple))))) #43 edges significant
print(len(list(set(integrated_edges).intersection(set(proteomic_influence_edge_tuple))))) #After integrating with the proteomic data, 170 significant

#8 edges are the same between 43 edges and 170 edges
list1 = list(set(metabolomic_edges).intersection(set(proteomic_influence_edge_tuple)))
list2 = list(set(integrated_edges).intersection(set(proteomic_influence_edge_tuple)))
len(list(set(list1).intersection(set(list2))))

### Testing the integrated only pathways

In [ ]:
integrated_only = list(set(integrated_edges) - (set(metabolomic_edges)|set(proteomic_edges)))
len(integrated_only)

Common pathway | Common pathway            =   47   <br>

Common pathway | Metabolite only pathway      =   81   <br>
Proteomic-influenced metabolomic pathway | (Any)  =  26    <br>

Common pathway | Protein only edge   =   668   <br>
Proteomic-influenced metabolomic pathway | (Any)  =  272    <br>

Pathway only in metabolomics | Pathway only in proteomics   =   676   <br>

False positives: <br>
Pathway only in metabolomics | Pathway only in metabolomics   =   95  <br>
Pathway only in metabolomics | Pathway only in metabolomics   =   95  <br>

New integrated pathways: <br>
New pathway | (Any)   =   669  <br>


In [ ]:
print(len(proteomic_only_pathway))
print(len(proteomic_pathway_with_onemetabolite))
print(len(metabolomic_only_pathway))
print(len(metabolomic_pathway_with_oneprotein))
print(len(both_omic))
print(len(new_pathway))

In [ ]:
#Pathways where both are common pathways

common_pathways = list(set(proteomic_pathways) & set(metabolomic_pathways))
len(common_pathways)

counter = 0 

for edges in integrated_only:
    pathway_pair = list(edges)
    if (pathway_pair[0] in common_pathways) and (pathway_pair[1] in common_pathways):
        #print(pathway_pair)
        counter += 1

print(counter)

In [ ]:
#Proteomic influenced metabolomic pair (i.e. pairs with one of the 47 common pathways and one metabolomic only pair)
print(len(list(set(proteomic_influence_edge_tuple).intersection(set(integrated_only)))))
#Metabolomic influenced proteomic pair (i.e. pairs with one of the 47 common pathways and one proteomic only pair)
print(len(list(set(metabolomic_influence_edge_tuple).intersection(set(integrated_only)))))

In [ ]:
proteomic_influence_edge_tuple

In [ ]:
list1 = list(set(metabolomic_influence_edge_tuple).intersection(set(integrated_only)))

counter = 0 

for edges in list1:
    pathway_pair = list(edges)
    if ((pathway_pair[0] in common_pathways) or (pathway_pair[1] not in common_pathways)):
        counter += 1
    if ((pathway_pair[1] in common_pathways) and (pathway_pair[0] not in common_pathways)):
        #print(pathway_pair)
        counter += 1

print(counter)

In [ ]:
#Pathways where there is only one common pathway 

common_pathways = list(set(proteomic_pathways) & set(metabolomic_pathways))
len(common_pathways)

counter = 0 

for edges in integrated_only:
    pathway_pair = list(edges)
    if ((pathway_pair[0] in common_pathways) and (pathway_pair[1] not in common_pathways)):
        counter += 1
    if ((pathway_pair[1] in common_pathways) and (pathway_pair[0] not in common_pathways)):
        #print(pathway_pair)
        counter += 1

print(counter)

In [ ]:
#Pathways where only one is a common pathway and the other is a metabolomics only pathway

common_pathways = list(set(proteomic_pathways) & set(metabolomic_pathways))
len(common_pathways)

counter = 0 

for edges in integrated_only:
    pathway_pair = list(edges)
    if ((pathway_pair[0] in common_pathways) and (pathway_pair[1] not in common_pathways) and (pathway_pair[1] in metabolomic_pathways)):
        counter += 1
    if ((pathway_pair[1] in common_pathways) and (pathway_pair[0] not in common_pathways) and (pathway_pair[0] in metabolomic_pathways)):
        #print(pathway_pair)
        counter += 1

print(counter)

In [ ]:
#Pathways where only one is a common pathway and the other is a proteomics only pathway

common_pathways = list(set(proteomic_pathways) & set(metabolomic_pathways))
len(common_pathways)

counter = 0 

for edges in integrated_only:
    pathway_pair = list(edges)
    if ((pathway_pair[0] in common_pathways) and (pathway_pair[1] not in common_pathways) and (pathway_pair[1] in proteomic_pathways)):
        counter += 1
    if ((pathway_pair[1] in common_pathways) and (pathway_pair[0] not in common_pathways) and (pathway_pair[0] in proteomic_pathways)):
        #print(pathway_pair)
        counter += 1

print(counter)

In [ ]:
#Pathway pairs where one is part of metabolomics and not proteomics and the other is vice versa

counter = 0 

for edges in integrated_only:
    pathway_pair = list(edges)
    if (pathway_pair[0] in metabolomic_pathways and pathway_pair[0] not in proteomic_pathways) and (pathway_pair[1] in proteomic_pathways and pathway_pair[1] not in metabolomic_pathways):
        counter += 1
    if (pathway_pair[0] in proteomic_pathways and pathway_pair[0] not in metabolomic_pathways) and (pathway_pair[1] in metabolomic_pathways and pathway_pair[1] not in proteomic_pathways):
        counter += 1
        
print(counter)

In [ ]:
#Probably false postives:  96 overall
#Proteomic influenced only (i.e. pairs where both pathways are only in the proteomic dataset)
print(len(list(set(proteomic_only_edge_tuple).intersection(set(integrated_only)))))
#Metabolomic influenced only (i.e. pairs where both pathways are only in the metabolomic dataset)
print(len(list(set(metabolomic_only_edge_tuple).intersection(set(integrated_only)))))

In [ ]:
#Check the significance of the 'false positives' IN THE INTEGRATED DATASET
proteomic_false_pos = list(set(proteomic_only_edge_tuple).intersection(set(integrated_only)))
metabolomic_false_pos = list(set(metabolomic_only_edge_tuple).intersection(set(integrated_only)))

In [ ]:
proteomic_false_pos_pval = []
metabolomic_false_pos_pval = []

for edge in proteomic_false_pos:
    if edge in list(p_val_dict.keys()):
        proteomic_false_pos_pval.append(p_val_dict[edge])

for edge in metabolomic_false_pos:
    if edge in list(p_val_dict.keys()):
        metabolomic_false_pos_pval.append(p_val_dict[edge])

proteomic_false_pos_pval[:100]
#len(proteomic_false_pos_pval)

print(metabolomic_false_pos_pval) #[0.000747091124962897] (not that close to threshold)
sum(proteomic_false_pos_pval)/len(proteomic_false_pos_pval)  #average = 0.0045  #close to p < 0.05 threshold

In [ ]:
#Pathway pairs where at least one is one of the 35 new pathways
 
counter = 0 

for edges in integrated_only:
    pathway_pair = list(edges)
    if (pathway_pair[0] in new_pathways) or (pathway_pair[1] in new_pathways):
        #print(pathway_pair[0])
        counter += 1

print(counter)

In [ ]:
#Pathway pairs where at least one is one of the 35 new pathways
 
counter = 0 

for edges in integrated_only:
    pathway_pair = list(edges)
    if (pathway_pair[1] in new_pathways) and (pathway_pair[0] in new_pathways):
        #print(pathway_pair[0])
        counter += 1

print(counter)

#38+23

In [ ]:
len(list(set(integrated_edge_tuple).intersection(set(integrated_only))))